### Application of $\chi^2$ on ASCAD

On a per-sample-point basis.

In [8]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from tools.nbloader import NotebookLoader

from tools.traceloader import ASCAD, TraceGroup

sns.set_style("whitegrid")

In [9]:
KEY_SIZE = 256
TRACE_LEN = 1400

ascad = ASCAD()
ascad.default.profile.traces.shape

(200000, 1400)

In [10]:
chi2 = NotebookLoader(["../tvla"]).load_module('chi2')

importing Jupyter notebook from ../tvla/chi2.ipynb


### chi-squared sliding window

In [11]:
SLIDING_WINDOW_SIZE = 10

def sliding_window(tg: TraceGroup, key_l=1, key_r=1, progress=True):
    trace_l = tg.profile.filter_traces(key_l)
    trace_r = tg.attack.filter_traces(key_r)

    trace_size = ASCAD.trace_len
    acc = np.zeros(trace_size - SLIDING_WINDOW_SIZE)

    window_ixs = range(0, trace_size - SLIDING_WINDOW_SIZE)
    if progress:
        window_ixs = tqdm(window_ixs)

    for ix in window_ixs:
        poi = (ix, ix + SLIDING_WINDOW_SIZE)
        p_value = 1 - chi2.chi_squared(chi2.count(trace_l, poi), chi2.count(trace_r, poi))
        acc[ix] = p_value

    return acc

SW_EQ = sliding_window(ascad.default)

100%|██████████| 1390/1390 [00:04<00:00, 280.77it/s]


In [12]:
SW_NEQ = sliding_window(ascad.default, 1, 2)

100%|██████████| 1390/1390 [00:04<00:00, 288.92it/s]


### Experiments with sliding window on $\chi^2$

With a sliding window over only 1, a

In [1]:
poi = NotebookLoader(["../data"]).load_module("poi")

NameError: name 'NotebookLoader' is not defined

Small $p$ values give evidence to reject the null hypothesis and conclude that the observations are not independent.

In [ ]:
ax = sns.lineplot(data=pd.DataFrame(poi.MOMENT_DIFFERENCES[1]), legend=None)
ax.set(ylabel = 'first order moment differences')
ax2 = plt.twinx()
ax2.set(ylabel = '$p$-values sliding window equal key')
sns.lineplot(data=pd.DataFrame([*([np.nan] * round(SLIDING_WINDOW_SIZE / 2)), *SW_EQ]), palette=[sns.color_palette()[1]], legend=None)

In [ ]:
ax = sns.lineplot(data=pd.DataFrame(poi.MOMENT_DIFFERENCES[1]), legend=None)
ax.set(ylabel = 'first order moment differences')
ax2 = plt.twinx()
ax2.set(ylabel = '$p$-values sliding window different key')
sns.lineplot(data=pd.DataFrame([*([np.nan] * round(SLIDING_WINDOW_SIZE / 2)), *SW_NEQ]), palette=[sns.color_palette()[1]], legend=None)

In [ ]:
SW_DIFF = SW_EQ - SW_NEQ
SW_DIFF_CUTOFF = SW_DIFF * (SW_DIFF > 0)

In [ ]:
ax = sns.lineplot(data=pd.DataFrame(poi.MOMENT_DIFFERENCES[1]), legend=None)
ax.set(ylabel = 'first order moment differences')
ax2 = plt.twinx()
ax2.set(ylabel = '$p$-value difference')
sns.lineplot(data=pd.DataFrame([*([np.nan] * round(SLIDING_WINDOW_SIZE / 2)), *SW_DIFF_CUTOFF]), palette=[sns.color_palette()[1]], legend=None)